# S-CASC

This notebook contains code for running S-CASC (see manuscript for method details). 

Prior to running S-CASC, you will need to:

- Compute co-activity scores (Step 0 in main folder, 'Co-activity scores' steps in peak_scores/ folder)
- Create peak categories/annotations (Step 1 in S-CASC folder)
- Compute number of nearby genes (Step 2 in S-CASC folder)
- Compute stratified co-accessibility score (Step 3 in S-CASC folder)

Once these steps are completed, specify input file names in the "Specify input files" section below, then run all blocks to implement S-CASC!

(Note: there is also code below for generating per-peak causal effect sizes from S-CASC to use in functionally informed fine-mapping. Run all blocks and then proceed to "Save per-peak causal effect sizes.")


## Load packages

In [1]:
import sys
sys.path.insert(1, "..")
import functions as fn
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
import pybedtools
import os
from glob import glob
import warnings
import seaborn as sns
warnings.filterwarnings("ignore", message="invalid value encountered in double_scalars")


## Functions

In [4]:
def data_files_to_data_frame(column_labels, files, merge_column):
    # Set up annotation DF
    df = pd.read_csv(files[0], sep = '\t')
    df.columns = [merge_column, column_labels[0]]
    for i in range(1, len(column_labels)):
        df_single = pd.read_csv(files[i], sep = '\t')
        df_single.columns = [merge_column, column_labels[i]]
        df = df.merge(df_single)
    return df

def load_coactivity_scores(coactivity_scores_file, feature):
    coactivity_scores_df = pd.read_csv(coactivity_scores_file, sep = '\t')
    coactivity_scores_df = coactivity_scores_df.rename(columns = {'neighbors': 'neighbors_coactivity'})
    return coactivity_scores_df[[feature, 'coactivity_score', 'neighbors_coactivity']]


def stratified_regression(annotations, annotation_files, coaccess_scores_files, coactivity_score_file,
                          outfile = None, force = False, merge_coactivity_how = 'left', feature = 'peak',
                         return_annot_df = False, verbose = False, 
                         covariate = None):
    if outfile != None and os.path.exists(outfile) and force == False:
        res = pd.read_csv(outfile, sep = '\t')
    else:
        # Reformat annotation names (if needed)
        annotations = [a.replace('.', '_') for a in annotations]
        
        # Create annotation and coaccessibility score dataframes
        annot_df = data_files_to_data_frame(annotations, annotation_files, feature)
        coaccess_scores_df = data_files_to_data_frame(annotations, coaccess_scores_files, feature)
        
        # Get coactivity scores and merge with dataframes
        coactivity_scores_df = load_coactivity_scores(coactivity_scores_file, feature)
        scores_df = coaccess_scores_df.merge(coactivity_scores_df, how = merge_coactivity_how)
        
        if verbose == True:
            print(len(annot_df))
            print("len(coaccess_scores_df):", len(coaccess_scores_df))
            print("len(coactivity_scores_df):", len(coactivity_scores_df))
            print("len(scores_df):", len(scores_df))
            print(scores_df.isna().sum())
            print(len(scores_df))
        
        scores_df = scores_df.fillna(0)
        annot_df = annot_df.merge(coactivity_scores_df, how = merge_coactivity_how)
        
        if verbose == True:
            print(annot_df.isna().sum())
            print(len(annot_df))
        
        annot_df = annot_df.fillna(0)
        
        # Create jackknife block column 
        if 'chr' not in annot_df.columns or 'chr' not in scores.columns:
            annot_df['chr'] = annot_df[feature].str.split("-").str[0]
            scores_df['chr'] = scores_df[feature].str.split("-").str[0]
        block_col = 'chr'

        # Get basic summary numbers
        num_annots = len(annotations)
        num_peaks = len(annot_df)
        num_blocks = annot_df[block_col].nunique()
        print("%s annotations (%s)" % (num_annots, ", ".join(annotations)))
        
        # Merge with covariate 
        if covariate_file != None:
            covariate = pd.read_csv(covariate_file, sep = '\t')
            covariate_name = covariate.columns.tolist()[1]
            scores_df = scores_df.merge(covariate)
        
        # Jackknife annotation sizes and standard deviations
        annot_sizes, annot_sizes_jackknife, m = fn.block_jackknife(
            annot_df, block_col, fn.get_annotation_size, num_estimands = num_annots, 
            held_out_estimates_only = True, annot_names = annotations)

        annot_sd, annot_sd_jackknife, m = fn.block_jackknife(
            annot_df, block_col, fn.get_annotation_sd, num_estimands = num_annots, 
            held_out_estimates_only = True, annot_names = annotations)

        # Compute tau + SE for each annotation
        tau_est, tau_se, tau_jackknife_est, m = fn.block_jackknife(
            scores_df, block_col, fn.get_coefficients, num_estimands = num_annots, predictors = annotations, covariates = [covariate_name])
        if num_annots == 1:
            tau_est, tau_se = np.array([tau_est]), np.array([tau_se])
        tau_pvals = [fn.get_p_from_est_and_se(tau, se) for tau, se in zip(tau_est, tau_se)]
        
        # Get h2 estimates for all annotations
        h2_est, junk, m = fn.block_jackknife(
            annot_df, block_col, fn.get_annotation_h2_multiple, num_estimands = num_annots, 
            annot_names = annotations, taus = tau_est, held_out_estimates_only = True)

        # Get h2 jackknife estimates, SE, p-value for all annotations (using jackknifed tau values)
        blocks = np.sort(annot_df[block_col].unique())
        h2_jackknife_est = np.zeros((len(blocks), num_annots))
        for i, block in enumerate(blocks):
            tmp = annot_df[annot_df[block_col] != block]
            h2_jackknife_est[i] = fn.get_annotation_h2_multiple(tmp, annot_names = annotations, taus = tau_jackknife_est[i,:])

        h2_se = np.zeros(num_annots)
        for i, annot_name in enumerate(annotations):
            junk, h2_se[i] = fn.get_weighted_jackknife_estimate_and_se(h2_est[i], h2_jackknife_est[:,i], m, num_peaks)
        h2_pvals = [fn.get_p_from_est_and_se(h2, se) for h2, se in zip(h2_est, h2_se)]

        # Get overall h2 estimate
        h2g, h2g_se, h2g_pval = h2_est[0], h2_se[0], h2_pvals[0]
        h2g_jackknife = h2_jackknife_est[:,0]

        # Get tau star
        tau_star_est = num_peaks * annot_sd * tau_est / h2g
        tau_star_jackknife_est = np.zeros((len(blocks), num_annots))
        for i in range(len(blocks)):
            tau_star_jackknife_est[i] = (num_peaks - m[i]) * annot_sd_jackknife[i] * tau_jackknife_est[i] / h2g_jackknife[i]

        tau_star_se = np.zeros(num_annots)
        for i, annot_name in enumerate(annotations):
            junk, tau_star_se[i] = fn.get_weighted_jackknife_estimate_and_se(tau_star_est[i], tau_star_jackknife_est[:,i], m, num_peaks)
        tau_star_pvals = [fn.get_p_from_est_and_se(tau_star, se) for tau_star, se in zip(tau_star_est, tau_star_se)]

        # Get h2 enrichment estimates
        enrich_est = (h2_est / h2g) / (annot_sizes / num_peaks)

        # Get h2 enrichment jackknife estimates, SE, p-value for all annotations
        enrich_jackknife_est = np.zeros((len(blocks), num_annots))
        for i in range(len(blocks)):
            enrich_jackknife_est[i] = (h2_jackknife_est[i] / h2g_jackknife[i]) / (annot_sizes_jackknife[i] / (num_peaks - m[i]))

        enrich_se = np.zeros(num_annots)
        for i, annot_name in enumerate(annotations):
            junk, enrich_se[i] = fn.get_weighted_jackknife_estimate_and_se(enrich_est[i], enrich_jackknife_est[:,i], m, num_peaks)
        enrich_pvals = [fn.get_p_from_est_and_se(enrich - 1, se) for enrich, se in zip(enrich_est, enrich_se)]

        res = pd.DataFrame({'annot': annotations,
                            'tau': tau_est,
                            'tau_se': tau_se,
                            'tau_p': tau_pvals,
                            'tau_star': tau_star_est,
                            'tau_star_se': tau_star_se,
                            'tau_star_p': tau_star_pvals,
                            'h2': h2_est,
                            'h2_se': h2_se,
                            'h2_p': h2_pvals,
                            'h2_enrich': enrich_est,
                            'h2_enrich_se': enrich_se,
                            'enrich_p': enrich_pvals,
                            'total_h2': [h2g] * num_annots,
                            'total_h2_se': [h2g_se] * num_annots,
                            'total_h2_p': [h2g_pval] * num_annots})
        if outfile != None:
            res.to_csv(outfile, sep = '\t', index = False)
    if return_annot_df == True:
        return res, annot_df
    else:
        return res

def plot_joint_h2_enrichments(in_res, annot_sizes_dict = None, sort = False, label_dict = None, 
                                     figsize = (4,5), xtick_rotation = 90, xtick_ha = 'center', axis = None, feature = "peak"):
    if axis == None:
        fig, axis = plt.subplots(figsize = figsize)
    res = res[(res['annot'] != 'all_%ss' % feature)]
    if sort == True:
        res = in_res.sort_values(by = 'h2_enrich', ascending = False)
    else:
        res = in_res.copy()
    labels = res['annot'].tolist()
    est = res['h2_enrich']
    se = res['h2_enrich_se']
    pvals = res['enrich_p']
    fn.plot_across_conditions_multi(est.tolist(), se.tolist(), labels, xlabel = "Peak category", ylabel = r"Causal effect size enrichment", pvals = pvals.tolist(), null = 1,
                                    xtick_rotation = xtick_rotation, xtick_ha = xtick_ha, figsize = figsize, axis = axis)
    ax.hlines(xmin = -1, xmax = len(labels), y = 1, color = 'darkgrey', ls = '--')
    

## Run regression

In [ ]:
# Specify input files
annotations = [] ### Fill in with names of peak categories (should match the annotation file names, e.g. */<annotation>.annot
annotation_files = [] ### Fill in with annotation files (created from create_annot.py)
coaccess_scores_files = [] ### Fill in with co-accessibility score files (created from coaccessibility_to_annotation_proximal.py)
coactivity_scores_file = "" ### Fill in with co-activity scores for each peak
covariate_file = "" ### Fill in with file denoting number of nearby genes per peak (output of XX)
outfile = "" ### Fill in with path to file to save results to


In [ ]:

num_annots = len(annotations)
print("%s annotations: %s" % (num_annots, annotations))

# Run regression
print("Running S-CASC!")
res, annot_df = stratified_regression(
    annotations, 
    annotation_files, 
    coaccess_scores_files, 
    coactivity_scores_file,
    covariate_file = covariate_file,
    feature = "peak", 
    return_annot_df = True
)

# Plot enrichments
fig, ax = plt.subplots(figsize = (4,5))
plot_joint_h2_enrichments(res[res['annot'] != 'all_peaks'], sort = True, axis = ax)

# Save output
res_out = res.rename(columns = {
    'annot': 'Category',
    'h2_enrich': 'Enrichment',
    'h2_enrich_se': 'Enrichment (s.e.)',
    'enrich_p': 'Enrichment (P-value)'}
               )[['Category', 'Enrichment', 'Enrichment (s.e.)', 'Enrichment (P-value)']]
    res_out = res_out[res_out['Category'] != 'all_peaks']
    res_out['Category'] = res_out['Category'].map(label_dict).fillna(res_out['Category'])
    res_out = res_out.sort_values(by = "Category")
    res_out.to_csv(outfile, sep = '\t', index = False)
    print("Wrote S-CASC output to %s!" % outfile)


### Save per-peak causal effect sizes (for functionally-informed fine-mapping (optional)

In [ ]:
# Run this block if you wish to perform functionally-informed fine-mapping
# Must run all above before running this block

weights_outfile = '%s.per_peak_causal_effec_size' % outfile

in_df = annot_df.copy()
annot_df_h2 = fn.get_annotation_per_peak_h2(in_df, annot_names = [a.replace('.', '_') for a in annotations], taus = res['tau'])
weights = annot_df_h2[[feature, 'per_peak_h2']]
weights['per_%s_h2' % feature] = np.maximum(weights['per_peak_h2'], 1e-10)
weights = weights[[feature, 'per_%s_h2' % feature]]
weights.to_csv(weights_outfile, sep = '\t', index = False)
print("Wrote functionally-informed priors to %s!" % weights_outfile)

scores_df, annot_df, tau_est, tau_se, tau_hoe, tau_pvals = stratified_regression(annotations, annotation_files, coaccess_scores_files, coactivity_scores_file,
                           outfile = outfile, force = True,
                           feature = feature                
)
